# SAN FRANCISCO BUILDING PERMITS

On this project,the San Francisco’s building permit public data create, clean and explore. 
Usinge OpenStreetMap API to find the geo coordinates of buildings and analyze the final results in Tableau. 

### Project Procedure: 
* Leverage Tableau to analyze a dataset based on San Francisco's buildings permits dataset to gain insights on the metropolitan are development.
* Gain data analytics experience exploring and filter the San Francisco's buildings permits data.Transform it using basic string cleaning techniques to generate a new clean dataset.
* Utilize R and RStudio to create a function to make requests to the OpenStreeMaps.org API using address from the San Francisco buildings permits dataset.


### Dataset Resource:

In many cases we have access to data with location information e.g. Address, City ,County ,Zip Code , Country. To visualize this data with more detail we need to use geolocation.
In order get the actual geolocation coordinates (Longitute,Latitude) for the location data, we need to calculate this coordenates.
Using nominatim a free search engine for OpenStreetMap data to easily find out geolocation coordinates.
OpenStreetMap: Is open data built by a community of mappers that contribute and maintain data about roads, trails, cafÃ©s, railway stations, and much more, all over the world.

https://www.openstreetmap.org/about

### Geocoding: OpenStreetMaps API Search Function

The geocode function below allow us to make requests to Nominatim a search engine for OpenStreetMap data.

Nominatim: https://nominatim.openstreetmap.org/
Resources: https://nominatim.org/release-docs/develop/api/Search

### Installing Required Packages:

In [1]:
install.packages("tidyverse")
library(tidyverse)

install.packages("lubridate")
library(lubridate)

install.packages("rvest")
library(rvest)

install.packages("stringi")
library(stringi)

install.packages("jsonlite")
library(jsonlite)

### Check if libraries are installed and search API URL of OpenStreetMap.

In [1]:
# INPUT LOCATION VARIABLES
# [RECORD_ID] , [ADDRESS], [CITY] , [STATE] , [ZIPCODE]

geocode <- function(record_id, address, city, state, zipcode){
 
   # NOMINATIM SEARCH API URL
  src_url <- "https://nominatim.openstreetmap.org/search?q="
  
  ####### INPUTS PREPARATION ##########
  
  city <- str_replace_all(string = city, pattern = "\\s|,", replacement = "+")
  
  # CREATE A FULL ADDRESS
  addr <- paste(address, city, state, zipcode, sep = "%2C")
 
   # CREATE A SEARCH URL BASED ON NOMINATIM API TO RETURN GEOJSON
  requests <- paste0(src_url, addr, "&format=geojson")
  
  # ITERATE OVER THE URLS AND MAKE REQUEST TO THE SEARCH API
  for (i in 1:length(requests)) {
    
    # MAKE HTML REQUEST TO API AND TRANSFORME HTML RESPONSE TO JSON
    response <- read_html(requests[i]) %>%
      html_node("p") %>%
      html_text() %>%
      fromJSON()
    
    # FROM THE RESPONSE EXTRACT LATITUDE AND LONGITUDE COORDINATES
    lon <- response$features$geometry$coordinates[[1]][1]
    lat <- response$features$geometry$coordinates[[1]][2]
    
    # CREATE A COORDINATES DATAFRAME
    if(i == 1) {
      loc <- tibble(record_id = record_id[i], 
                    address = str_replace_all(addr[i], "%2C", ","),
                    latitude = lat, longitude = lon)
    }else{
      df <- tibble(record_id = record_id[i], 
                   address = str_replace_all(addr[i], "%2C", ","),
                   latitude = lat, longitude = lon)
      loc <- bind_rows(loc, df)
    }
  }
  return(loc)
}


### Data Source: San Francisco Buildings Permit Data
This data set pertains to all types of structural permits. Data includes details on application/permit numbers, job addresses, supervisorial districts, and the current status of the applications.

Source: https://data.sfgov.org/Housing-and-Buildings/Building-Permits/i98e-djp9


In [2]:
## Reading Dataset

data <- read.csv("data/building_permits.csv")

In [3]:
## Reading the first few rows of the dataset
head(data)

In [ ]:
## Reading the last few rows of the dataset
tail(data)

In [ ]:
## Reading column names
colnames(data)

#### Lets change the name of some columns and remote other that are irrelevant for this use case.

In [ ]:
## enter the columns in the varialble which is not needed for further analysis

colremove = c("First Construction Document Date",
              "Structural Notification",
              "Number of Existing Stories",
              "Number of Proposed Stories",
              "Voluntary Soft-Story Retrofit",
              "Fire Only Permit","Existing Units",
              "Proposed Units","Plansets",
              "TIDF Compliance","Existing Construction Type",
              "Proposed Construction Type","Site Permit",
              "Supervisor District","Current Police Districts",
              "Current Supervisor Districts",
              "Current Status Date", "Permit Creation Date",
              "Analysis Neighborhoods","Lot","Location",
              "SF Find Neighborhoods","Unit","Block", "Permit Type",
              "Unit Suffix","Street Number Suffix",
              "Existing Construction Type Description")

## remove the extra columns.
data <- data[colnames(data)[1:47]] %>% select(-all_of(colremove))

## rename few columns 
data <- rename(data, "Neighborhoods Boundaries" = "Neighborhoods - Analysis Boundaries")

data <- rename(data, "Permit Type" = "Permit Type Definition")

#### Lower case all the column names

In [ ]:
colnames(data ) <- str_replace_all(tolower(colnames(data))," ","_")

#### Checking the Dimensions of the data table

In [ ]:
dim_desc(data)

#### Filter out data with current status column= 'complete'

In [ ]:
bldg_df <- data %>% filter(current_status == "complete")

head(bldg_df)

#### Again checking for the data dimensions

In [ ]:
dim_desc(bldg_df)

### Data Preparation and Cleaning

In [ ]:
bldg_df <- na.omit(bldg_df)

dim_desc(bldg_df)

### String Cleaning and Pattern Replacement

In [ ]:
bldg_df$city <-"San Francisco"
bldg_df$state <- "CA"
bldg_df$record_id <- as.character(bldg_df$record_id)

bldg_df$proposed_construction_type_description <- 
  bldg_df$proposed_construction_type_description %>%
  str_remove_all(pattern = "\\(|\\)") %>%
  str_replace_all(" ", "_")%>%
  as_factor()

bldg_df$street_name <- str_remove(string = bldg_df$street_name,pattern = "^0+")
                                
bldg_df$street_name <- str_replace_all(string = bldg_df$street_name,pattern ="Bay Shore", replacement = "Bayshore")

bldg_df$street_suffix <- str_replace_all(string = bldg_df$street_suffix, pattern = "Bl", replacement = "Blvd")

bldg_df$street_suffix <- str_replace_all(string = bldg_df$street_suffix, pattern = "Tr", replacement = "Ter")

bldg_df$changes_use <- bldg_df$existing_use != bldg_df$proposed_use
bldg_df$estimated_revised_diff <-bldg_df$revised_cost -
  bldg_df$estimated_cost

### Data Types

In [ ]:
bldg_df$issued_date <- bldg_df$issued_date %>% mdy() %>% as_date()
bldg_df$issued_year <- bldg_df$issued_date %>% year()
bldg_df$issued_month <- bldg_df$issued_date %>% month(label = TRUE)
bldg_df$completed_date <- bldg_df$completed_date %>% mdy() %>% as_date()
bldg_df$completed_year <- bldg_df$completed_date %>% year()
bldg_df$completed_month <- bldg_df$completed_date %>% month(label = TRUE)

bldg_df$issued_completed_days <- bldg_df$completed_date - bldg_df$issued_date 
bldg_df$issued_completed_years <- bldg_df$completed_year - bldg_df$issued_year 

bldg_df$filed_date <- bldg_df$filed_date %>% mdy() %>% as_date()
bldg_df$filed_issued_days <- bldg_df$issued_date - bldg_df$filed_date

bldg_df$permit_expiration_date <- bldg_df$permit_expiration_date %>% mdy() %>% as_date()
bldg_df$issued_expiration_days <- bldg_df$permit_expiration_date - bldg_df$issued_date

### Project Cost and Zoning
To look deeper into changes in zoning and type of constructions. We want to select only project that changed the building use. Also we will like to look at developments with significant capital investment in particular greater than half a million dollars

In [ ]:
bldg_df <- bldg_df %>% arrange(desc(estimated_cost))
bldg_df <- bldg_df[bldg_df$changes_use == TRUE,]
bldg_df <- bldg_df %>% filter(revised_cost > 500000)

bldg_df$zipcode <- as_factor(bldg_df$zipcode)
bldg_df$permit_type <- as_factor(bldg_df$permit_type)
bldg_df$neighborhoods_boundaries <- as_factor(bldg_df$neighborhoods_boundaries)
bldg_df$proposed_use <- as_factor(bldg_df$proposed_use)

head(bldg_df)

### Final Dimensions of Dataset

In [ ]:
dim_desc(bldg_df)

### Preparing the address string variable to match the OpenStreeMaps API

In [ ]:
# ADDRESS VARIABLE MUST MATH NOMINATIM API
address <- paste0(bldg_df$street_number," ",
                  bldg_df$street_name," ",
                  bldg_df$street_suffix)

# DATA CLEANING SPECIAL CASES (e.g. 3333-3339 N CLARK)
query <- stri_replace(str = address, 
                      replacement = "", 
                      regex = "(-[0-9]+\\s)")

# REPLACE SPACES (\\s) OR COMMAS (,) WITH + SIGN
query <- str_replace_all(string = query, 
                         pattern = "\\s|,", 
                         replacement = "+")

head(query)

### Data Collection: OpenStreeMaps API Request

In [ ]:
df <- geocode (record_id = bldg_df$record_id,
              address = query,
              city = bldg_df$city,
              state = bldg_df$state,
              zipcode = bldg_df$zipcode)

Finally lets save the new dataset containing the restaurants Geolocation (latitude, longitude).

In [ ]:
write_csv(bldg_df, "data/building_geocoord.csv")

Further, Tableau is used to visualize the data and map out all charts related to the dataset. 